In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import mlflow

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import classification_report, accuracy_score

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from tqdm import tqdm
import os

In [2]:
wd = os.getcwd().split("/")
repo_i = wd.index("ml-ops")
repo_path = "/".join(wd[:repo_i+1])
repo_path

'/Users/emmajuansalazar/Documents/spring-mod-2/ml-ops'

# The Data

**Exploring Predictive Health Factors**

[Kaggle Competition](https://www.kaggle.com/competitions/exploring-predictive-health-factors/data?select=train.csv)

Polycystic ovary syndrome (PCOS) is a common endocrine disorder that affects the reproductive health and quality of life of millions of women worldwide. Research has shown that lifestyle choices such as diet, exercise, stress management, etc. have a significant impact on the pathogenesis and symptom relief of PCOS. However, there are still many gaps in research on the relationship between lifestyle and PCOS, especially in terms of individual differences and comprehensive lifestyle interventions.

In order to promote research progress in this field, we are organizing the "Polycystic Ovary Syndrome (PCOS) Lifestyle Impact Research Competition". The aim is to use data-driven methods to deeply analyze the impact of lifestyle choices on PCOS, and provide scientific basis for clinical intervention and health management.

In [3]:
pcos = pd.read_csv(f"{repo_path}/data/pcos/train.csv")

In [4]:
pcos.head()

,ID,Age,Weight_kg,PCOS,Hormonal_Imbalance,Hyperandrogenism,Hirsutism,Conception_Difficulty,Insulin_Resistance,Exercise_Frequency,Exercise_Type,Exercise_Duration,Sleep_Hours,Exercise_Benefit
0,0,20-25,64.0,No,No,No,No,No,No,Rarely,"Cardio (e.g., running, cycling, swimming)",30 minutes,Less than 6 hours,Somewhat
1,1,15-20,55.0,No,No,No,No,No,No,6-8 Times a Week,No Exercise,Less than 30 minutes,6-8 hours,Somewhat
2,2,15-20,91.0,No,No,No,Yes,No,No,Rarely,"Cardio (e.g., running, cycling, swimming)",Less than 30 minutes,6-8 hours,Somewhat
3,3,15-20,56.0,No,No,No,No,No,No,6-8 Times a Week,"Cardio (e.g., running, cycling, swimming)",45 minutes,6-8 hours,Not at All
4,4,15-20,47.0,No,Yes,No,No,No,No,Rarely,No Exercise,Not Applicable,6-8 hours,Not Much


In [5]:
pcos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     210 non-null    int64  
 1   Age                    209 non-null    object 
 2   Weight_kg              208 non-null    float64
 3   PCOS                   210 non-null    object 
 4   Hormonal_Imbalance     208 non-null    object 
 5   Hyperandrogenism       207 non-null    object 
 6   Hirsutism              205 non-null    object 
 7   Conception_Difficulty  209 non-null    object 
 8   Insulin_Resistance     209 non-null    object 
 9   Exercise_Frequency     208 non-null    object 
 10  Exercise_Type          208 non-null    object 
 11  Exercise_Duration      208 non-null    object 
 12  Sleep_Hours            208 non-null    object 
 13  Exercise_Benefit       209 non-null    object 
dtypes: float64(1), int64(1), object(12)
memory usage: 23.1+ KB

In [6]:
pcos = pcos.set_index("ID")
pcos = pcos.iloc[:,[1,0] + list(range(2,13))]

In [7]:
pcos.shape

(210, 13)

In [8]:
plt.figure(figsize=(8, 5))
plt.hist(pcos['Weight_kg'], bins=20)
plt.title('Weight Distribution')
plt.xlabel('Weight')
plt.ylabel('Frequency')
plt.show()

In [9]:
for col in pcos.iloc[:, 1:].columns:
    counts = pcos.groupby(col).size().reset_index()
    display(counts)

,Age,0
0,15-20,50
1,20-25,125
2,25-30,4
3,30-25,1
4,30-35,2
5,30-40,1
6,35-44,4
7,45 and above,3
8,Less than 20,18
9,Less than 20-25,1


,PCOS,0
0,No,164
1,Yes,46


,Hormonal_Imbalance,0
0,No,95
1,"No, Yes, not diagnosed by a doctor",1
2,Yes,111
3,Yes Significantly,1


,Hyperandrogenism,0
0,No,175
1,Yes,32


,Hirsutism,0
0,No,148
1,"No, Yes, not diagnosed by a doctor",2
2,Yes,55


...

,Exercise_Frequency,0
0,1-2 Times a Week,35
1,3-4 Times a Week,23
2,6-8 Times a Week,17
3,6-8 hours,1
4,Less than 6 hours,1
5,Less than usual,1
6,Never,28
7,Rarely,102


,Exercise_Type,0
0,Cardio (e.g.,25
1,"Cardio (e.g., running, cycling, swimming)",51
2,"Cardio (e.g., running, cycling, swimming), Fle...",4
3,"Cardio (e.g., running, cycling, swimming), None",1
4,"Cardio (e.g., running, cycling, swimming), Str...",4
5,"Cardio (e.g., running, cycling, swimming), Str...",2
6,Flexibility and balance (e.g.,1
7,"Flexibility and balance (e.g., yoga, pilates)",16
8,"Flexibility and balance (e.g., yoga, pilates),...",1
9,High-intensity interval training (HIIT),3


,Exercise_Duration,0
0,20 minutes,1
1,30 minutes,33
2,30 minutes to 1 hour,1
3,45 minutes,17
4,Less than 30 minutes,62
5,Less than 6 hours,1
6,More than 30 minutes,7
7,Not Applicable,86


,Sleep_Hours,0
0,3-4 hours,1
1,6-8 hours,135
2,9-12 hours,13
3,Less than 6 hours,58
4,More than 12 hours,1


,Exercise_Benefit,0
0,Not Much,21
1,Not at All,26
2,Somewhat,137
3,Yes Significantly,25


## Fixing category mismatches

In [10]:
# Age
pcos.loc[(pcos["Age"] == "Less than 20") | (pcos["Age"] == "Less than 20-25"), "Age"] = "15-20"

# Hormonal_Imbalance
pcos.loc[pcos["Hormonal_Imbalance"] == "Yes Significantly", "Hormonal_Imbalance"] = "Yes"
pcos.loc[pcos["Hormonal_Imbalance"] == "No, Yes, not diagnosed by a doctor", "Hormonal_Imbalance"] = "Unknown"

# Hirsutism
pcos.loc[pcos["Hirsutism"] == "No, Yes, not diagnosed by a doctor", "Hirsutism"] = "Unknown"

# Conception_Difficulty
pcos.loc[pcos["Conception_Difficulty"] == "Yes, diagnosed by a doctor", "Conception_Difficulty"] = "Yes"
pcos.loc[pcos["Conception_Difficulty"] == "No, Yes, not diagnosed by a doctor", "Conception_Difficulty"] = "Unknown"

# Insulin_Resistance
pcos.loc[pcos["Insulin_Resistance"] == "No, Yes, not diagnosed by a doctor", "Insulin_Resistance"] = "Unknown"

# Exercise_Frequency
pcos.loc[pcos["Exercise_Frequency"].isin(["6-8 hours", "Less than 6 hours", "Less than usual"]), "Exercise_Frequency"] = "1-2 Times a Week"

# Exercise_Type
pcos["Exercise_Type"] = pcos["Exercise_Type"].fillna("Unknown")
pcos.loc[pcos["Exercise_Type"].str.startswith("Cardio"), "Exercise_Type"] = "Cardio"
pcos.loc[pcos["Exercise_Type"].str.startswith("Flexibility"), "Exercise_Type"] = "Flexibility and balance"
pcos.loc[pcos["Exercise_Type"].str.startswith("Strength"), "Exercise_Type"] = "Strength training"
pcos.loc[pcos["Exercise_Type"] == "Somewhat", "Exercise_Type"] = "Unknown"

# Exercise_Duration
pcos.loc[pcos["Exercise_Duration"] == "20 minutes", "Exercise_Duration"] = "Less than 30 minutes"
pcos.loc[pcos["Exercise_Duration"] == "30 minutes to 1 hour", "Exercise_Duration"] = "More than 30 minutes"
pcos.loc[pcos["Exercise_Duration"] == "45 minutes", "Exercise_Duration"] = "More than 30 minutes"
pcos.loc[pcos["Exercise_Duration"] == "Less than 6 hours", "Exercise_Duration"] = "Not Applicable"

# Sleep Hours
pcos.loc[pcos["Sleep_Hours"] == "3-4 hours", "Sleep_Hours"] = "Less than 6 hours"

# Target Variable: PCOS
pcos.loc[pcos["PCOS"] == "Yes", "PCOS"] = 1
pcos.loc[pcos["PCOS"] == "No", "PCOS"] = 0


In [11]:
for col in pcos.iloc[:, 1:].columns:
    counts = pcos.groupby(col).size().reset_index()
    display(counts)

,Age,0
0,15-20,69
1,20-25,125
2,25-30,4
3,30-25,1
4,30-35,2
5,30-40,1
6,35-44,4
7,45 and above,3


,PCOS,0
0,0,164
1,1,46


,Hormonal_Imbalance,0
0,No,95
1,Unknown,1
2,Yes,112


,Hyperandrogenism,0
0,No,175
1,Yes,32


,Hirsutism,0
0,No,148
1,Unknown,2
2,Yes,55


...

,Exercise_Frequency,0
0,1-2 Times a Week,38
1,3-4 Times a Week,23
2,6-8 Times a Week,17
3,Never,28
4,Rarely,102


,Exercise_Type,0
0,Cardio,87
1,Flexibility and balance,18
2,High-intensity interval training (HIIT),3
3,No Exercise,90
4,Strength training,9
5,Unknown,3


,Exercise_Duration,0
0,30 minutes,33
1,Less than 30 minutes,63
2,More than 30 minutes,25
3,Not Applicable,87


,Sleep_Hours,0
0,6-8 hours,135
1,9-12 hours,13
2,Less than 6 hours,59
3,More than 12 hours,1


,Exercise_Benefit,0
0,Not Much,21
1,Not at All,26
2,Somewhat,137
3,Yes Significantly,25


## Missing value inputation

Since we have at most 5 missing values in any which column, let's inpute the missing values with the mode/mean.

In [12]:
pcos.isnull().sum()

Weight_kg                2
Age                      1
PCOS                     0
Hormonal_Imbalance       2
Hyperandrogenism         3
Hirsutism                5
Conception_Difficulty    1
Insulin_Resistance       1
Exercise_Frequency       2
Exercise_Type            0
Exercise_Duration        2
Sleep_Hours              2
Exercise_Benefit         1
dtype: int64

In [13]:
float(pcos["Weight_kg"].mean())

56.159615384615385

In [14]:
# Mean for the Weight Column
mean_weight = float(pcos["Weight_kg"].mean())
pcos["Weight_kg"] = pcos["Weight_kg"].fillna(mean_weight)

for col in pcos.iloc[:, 1:].columns:
    mode_value = pcos[col].mode()[0]
    pcos[col] = pcos[col].fillna(mode_value)

/var/folders/bn/8l9zg4092yg1r262fl2zn9nw0000gn/T/ipykernel_44243/3599572613.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pcos[col] = pcos[col].fillna(mode_value)


In [15]:
pcos.isnull().sum()

Weight_kg                0
Age                      0
PCOS                     0
Hormonal_Imbalance       0
Hyperandrogenism         0
Hirsutism                0
Conception_Difficulty    0
Insulin_Resistance       0
Exercise_Frequency       0
Exercise_Type            0
Exercise_Duration        0
Sleep_Hours              0
Exercise_Benefit         0
dtype: int64

In [16]:
pcos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 210 entries, 0 to 209
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Weight_kg              210 non-null    float64
 1   Age                    210 non-null    object 
 2   PCOS                   210 non-null    int64  
 3   Hormonal_Imbalance     210 non-null    object 
 4   Hyperandrogenism       210 non-null    object 
 5   Hirsutism              210 non-null    object 
 6   Conception_Difficulty  210 non-null    object 
 7   Insulin_Resistance     210 non-null    object 
 8   Exercise_Frequency     210 non-null    object 
 9   Exercise_Type          210 non-null    object 
 10  Exercise_Duration      210 non-null    object 
 11  Sleep_Hours            210 non-null    object 
 12  Exercise_Benefit       210 non-null    object 
dtypes: float64(1), int64(1), object(11)
memory usage: 23.0+ KB


In [17]:
pcos = pcos.iloc[:, [0, 1] + list(range(3,13)) + [2]]

In [18]:
for col in pcos.iloc[:, 1:12].columns:
    pcos[col] = pcos[col].astype('category')

In [19]:
pcos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 210 entries, 0 to 209
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Weight_kg              210 non-null    float64 
 1   Age                    210 non-null    category
 2   Hormonal_Imbalance     210 non-null    category
 3   Hyperandrogenism       210 non-null    category
 4   Hirsutism              210 non-null    category
 5   Conception_Difficulty  210 non-null    category
 6   Insulin_Resistance     210 non-null    category
 7   Exercise_Frequency     210 non-null    category
 8   Exercise_Type          210 non-null    category
 9   Exercise_Duration      210 non-null    category
 10  Sleep_Hours            210 non-null    category
 11  Exercise_Benefit       210 non-null    category
 12  PCOS                   210 non-null    int64   
dtypes: category(11), float64(1), int64(1)
memory usage: 9.2 KB


# Dataset split

In [20]:
X = pcos.drop(columns="PCOS")
y = pcos["PCOS"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

# Modeling

We'll use three strategies:

- Linear Regression
- Decision Tree Classifier
- Random Forest Classifier

In [21]:
mlflow.set_tracking_uri('https://mlops-25-634030133521.us-west2.run.app')
mlflow.set_experiment('pcos-experiments')

2025/04/17 14:29:22 INFO mlflow.tracking.fluent: Experiment with name 'pcos-experiments' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1744925362472, experiment_id='1', last_update_time=1744925362472, lifecycle_stage='active', name='pcos-experiments', tags={}>

Run

`mlflow ui --backend-store-uri sqlite:///mlflow.db`

to open the mlflow UI.

In [22]:
# Feature groups
numerical_features = ["Weight_kg"]
categorical_features = [col for col in X.columns if col != "Weight_kg"]

# Preprocessing pipelines
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop="first", handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

In [23]:
def run_experiments(pipeline, grid, title):
    for i, params in tqdm(enumerate(grid)):
        with mlflow.start_run(run_name=f"{title} - Combo {i}"):
            try:
                # Fit the model
                pipeline.set_params(**params)
                pipeline.fit(X_train, y_train)

                # Evaluate on test set
                y_pred = pipeline.predict(X_test)
                test_accuracy = accuracy_score(y_test, y_pred)

                # Log parameters and metrics
                mlflow.log_params(params)
                mlflow.log_metric("test_accuracy", test_accuracy)

                # Log model
                # Create a single-row input example (after preprocessing)
                sample_input = X_test.iloc[[0]]  # wrap in double brackets to keep as DataFrame
                # Log the model with example and signature auto-inferred
                mlflow.sklearn.log_model(
                    sk_model=pipeline,
                    artifact_path="model",
                    input_example=sample_input,
                    signature=mlflow.models.infer_signature(sample_input, pipeline.predict(sample_input))
                )
            except Exception as e:
                mlflow.set_tag("run_status", "failed")
                mlflow.log_params(params)
                mlflow.set_tag("error_message", str(e))

In [24]:
def run_experiments_w_feature_selection(pipeline, grid, title):
    for i, params in tqdm(enumerate(grid)):
        with mlflow.start_run(run_name=f"{title} - Combo {i}"):
            try:
                # Fit the model
                pipeline.set_params(**params)
                pipeline.fit(X_train, y_train)

                # Evaluate on test set
                y_pred = pipeline.predict(X_test)
                test_accuracy = accuracy_score(y_test, y_pred)

                # Log parameters and metrics
                mlflow.log_params(params)
                mlflow.log_metric("test_accuracy", test_accuracy)

                # Log model
                # Create a single-row input example (after preprocessing)
                sample_input = X_test.iloc[[0]]  # wrap in double brackets to keep as DataFrame
                # Log the model with example and signature auto-inferred
                mlflow.sklearn.log_model(
                    sk_model=pipeline,
                    artifact_path="model",
                    input_example=sample_input,
                    signature=mlflow.models.infer_signature(sample_input, pipeline.predict(sample_input))
                )
                # Log selected feature names
                preprocessor = pipeline.named_steps["preprocessor"]
                feature_selector = pipeline.named_steps["feature_selection"]

                # Get full feature names
                ohe = preprocessor.named_transformers_["cat"]
                cat_feature_names = ohe.get_feature_names_out(categorical_features)
                all_feature_names = numerical_features + list(cat_feature_names)

                # Mask of selected features
                selected_mask = feature_selector.get_support()
                selected_features = [name for name, selected in zip(all_feature_names, selected_mask) if selected]

                # Log to MLflow
                mlflow.log_text("\n".join(selected_features), artifact_file="selected_features.txt")
        
            except Exception as e:
                
                mlflow.set_tag("run_status", "failed")
                mlflow.log_params(params)
                mlflow.set_tag("error_message", str(e))

## 1. Logistic Regression

In [25]:
# Classifier pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000, random_state=42))
])

# Hyperparameter grid
param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10],
    'classifier__penalty': ['l1', 'l2', None],
    'classifier__solver': ['lbfgs', 'liblinear', 'saga']
}

# Parameter Grid
grid = ParameterGrid(param_grid)

# Running the different experiments
run_experiments(pipeline, grid, "Logistic Regression")

1it [00:00,  1.10it/s]

🏃 View run Logistic Regression - Combo 0 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/b1319f6b0f264462b78ac2f9f93c2bb2
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


2it [00:07,  4.48s/it]

🏃 View run Logistic Regression - Combo 1 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/2c636891323a455493675a41959c490c
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


3it [00:12,  4.34s/it]

...

🏃 View run Logistic Regression - Combo 33 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/4667d7ce1c924c9cbe60bf04ddbd0ddf
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


35it [02:05,  2.98s/it]

🏃 View run Logistic Regression - Combo 34 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/d8c8cdca88cb4d2da31a8640c072eb72
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


36it [02:09,  3.61s/it]

🏃 View run Logistic Regression - Combo 35 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/40b5d4bc473a467bbc36e9d33fbbbed2
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


### With Feature selection

In [26]:
feature_selector = SelectFromModel(LogisticRegression(penalty="l1", solver="liblinear", random_state=42))

# Classifier pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("feature_selection", feature_selector),
    ("classifier", LogisticRegression(max_iter=1000, random_state=42))
])

# Hyperparameter grid
param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10],
    'classifier__penalty': ['l1'],
    'classifier__solver': ['lbfgs', 'liblinear'],
    "feature_selection__threshold": ["mean", "median", 0.01]
}

# Parameter Grid
grid = ParameterGrid(param_grid)

# Running the different experiments
run_experiments_w_feature_selection(pipeline, grid, "Logistic Regression + Feature Selection")

1it [00:00,  1.92it/s]

🏃 View run Logistic Regression + Feature Selection - Combo 0 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/3376f2b3e57f41a2b1a9c1eae34e8257
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


2it [00:00,  2.26it/s]

🏃 View run Logistic Regression + Feature Selection - Combo 1 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/92bcbd0c8ad446fb850b3d7fd87c69f8
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


3it [00:01,  2.58it/s]

...

🏃 View run Logistic Regression + Feature Selection - Combo 21 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/e4a89a901e684a809d387d43bf29cf5e
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


23it [00:57,  3.13s/it]

🏃 View run Logistic Regression + Feature Selection - Combo 22 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/b1296382b6be4551883660872ad4aa57
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


24it [01:02,  2.59s/it]

🏃 View run Logistic Regression + Feature Selection - Combo 23 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/267eb1cc02284518b278875ecd9846c8
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


## 2. Decision Tree Classifier

In [27]:
# Classifier pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", DecisionTreeClassifier(random_state=42))
])

# Hyperparameter grid
param_grid = {
    "classifier__max_depth": [3, 5, 10, None],
    "classifier__min_samples_split": [2, 5, 10],
    "classifier__criterion": ["gini", "entropy"]
}

# Parameter Grid
grid = ParameterGrid(param_grid)

# Running the different experiments
run_experiments(pipeline, grid, "Decision Tree")

1it [00:03,  3.76s/it]

🏃 View run Decision Tree - Combo 0 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/48814fba18bf43e49278e817e374466a
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


2it [00:07,  3.98s/it]

🏃 View run Decision Tree - Combo 1 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/72591dc437c34e379d23ee8df2792f7d
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


3it [00:12,  4.08s/it]

...

🏃 View run Decision Tree - Combo 21 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/850edafb975340e08dea09ec206ae2db
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


23it [01:37,  4.27s/it]

🏃 View run Decision Tree - Combo 22 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/6f717d35758845a189c8202e9b56e0d2
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


24it [01:41,  4.22s/it]

🏃 View run Decision Tree - Combo 23 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/cc75699dbb53439d81dafb50e5d5c6ce
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


### With Feature selection

In [28]:
feature_selector = SelectFromModel(DecisionTreeClassifier(random_state=42))

# Classifier pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("feature_selection", feature_selector),
    ("classifier", DecisionTreeClassifier(random_state=42))
])

# Hyperparameter grid
param_grid = {
    "feature_selection__threshold": ["mean", "median", 0.01],
    "classifier__max_depth": [None, 5, 10],
    "classifier__min_samples_split": [2, 5, 10],
    "classifier__criterion": ["gini", "entropy"]
}

# Parameter Grid
grid = ParameterGrid(param_grid)

# Running the different experiments
run_experiments_w_feature_selection(pipeline, grid, "Decision Tree + Feature Selection")

0it [00:00, ?it/s]

🏃 View run Decision Tree + Feature Selection - Combo 0 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/cff58eaf3bde4d8e9744628c2910fcb9
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


2it [00:09,  4.59s/it]

🏃 View run Decision Tree + Feature Selection - Combo 1 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/221e61b237b74d19a6a56cf61518bba2
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


3it [00:13,  4.62s/it]

...

🏃 View run Decision Tree + Feature Selection - Combo 51 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/2d2d82d04ade4d579f113d378627c5b0
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


53it [04:10,  4.81s/it]

🏃 View run Decision Tree + Feature Selection - Combo 52 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/b3b4ad3ed626474fbc62cf7a19551c17
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


54it [04:15,  4.73s/it]

🏃 View run Decision Tree + Feature Selection - Combo 53 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/e0aeacf5c97c48a5ba68f429b34373c4
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


## 3. Random Forest Classifier

In [29]:
# Classifier pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

# Hyperparameter grid
param_grid = {
    "classifier__n_estimators": [100, 200],
    "classifier__max_depth": [None, 5, 10],
    "classifier__min_samples_split": [2, 5],
    "classifier__criterion": ["gini", "entropy"]
}

# Parameter Grid
grid = ParameterGrid(param_grid)

# Running the different experiments
run_experiments(pipeline, grid, "Random Forest")

1it [00:04,  4.23s/it]

🏃 View run Random Forest - Combo 0 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/068ff51c32ff49b29994399caa5a1c4b
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


2it [00:08,  4.49s/it]

🏃 View run Random Forest - Combo 1 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/a865093f91164f03aaf8e82bed0d1651
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


3it [00:13,  4.36s/it]

...

🏃 View run Random Forest - Combo 21 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/6a7820c9aa934fc589b8245f6e673970
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


23it [01:40,  4.39s/it]

🏃 View run Random Forest - Combo 22 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/3873b0562c4846b6ba4c95df91cb7e49
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


24it [01:45,  4.39s/it]

🏃 View run Random Forest - Combo 23 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/632b9254c3fe4efa81ff708ffc4c0c43
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


### With Feature selection

In [30]:
feature_selector = SelectFromModel(RandomForestClassifier(n_estimators=200, random_state=42))

# Classifier pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("feature_selection", feature_selector),
    ("classifier", DecisionTreeClassifier(random_state=42))
])

# Hyperparameter grid
param_grid = {
    "feature_selection__threshold": ["mean", "median", 0.01],
    "classifier__n_estimators": [100, 200],
    "classifier__max_depth": [None, 5, 10],
    "classifier__min_samples_split": [2, 5],
    "classifier__criterion": ["gini", "entropy"]
}

# Parameter Grid
grid = ParameterGrid(param_grid)

# Running the different experiments
run_experiments_w_feature_selection(pipeline, grid, "Decision Tree + Feature Selection")

1it [00:00,  2.45it/s]

🏃 View run Decision Tree + Feature Selection - Combo 0 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/db47d875fc07460cbf4e8789fcfc98de
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


2it [00:00,  2.37it/s]

🏃 View run Decision Tree + Feature Selection - Combo 1 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/a21551a7c2cd4c9098133a712c4bac93
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


3it [00:01,  2.27it/s]

...

🏃 View run Decision Tree + Feature Selection - Combo 69 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/0ca7a435aff44382afb57ff00f2dc3e9
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


71it [00:28,  2.53it/s]

🏃 View run Decision Tree + Feature Selection - Combo 70 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/8115c5a2c1fc4e3f94aee0b95c2ce7b4
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


72it [00:29,  2.47it/s]

🏃 View run Decision Tree + Feature Selection - Combo 71 at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/2ffffd8b57bd46e9b0374e6911799863
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1


# The best model

In [32]:
# Replace this with your actual run ID
run_id = "267eb1cc02284518b278875ecd9846c8"
logged_model_uri = f"runs:/{run_id}/model"

# Load the model
model = mlflow.sklearn.load_model(logged_model_uri)

# Run it on your test set
y_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)

with mlflow.start_run(run_name="Final Model Evaluation"):
    mlflow.set_tag("source_run_id", run_id)
    mlflow.log_metric("final_test_accuracy", test_accuracy)

    # Optionally re-log the model to staging
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        input_example=X_test.iloc[[0]],
        signature=mlflow.models.infer_signature(X_test.iloc[[0]], model.predict(X_test.iloc[[0]])),
        registered_model_name="PCOS_LogReg_Final"
    )

    print("Loaded model from run ID and logged final test result!")
    print("Test Accuracy:", test_accuracy)

/opt/homebrew/anaconda3/envs/mlops/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Successfully registered model 'PCOS_LogReg_Final'.
2025/04/17 14:43:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: PCOS_LogReg_Final, version 1
Created version '1' of model 'PCOS_LogReg_Final'.


Loaded model from run ID and logged final test result!
Test Accuracy: 0.7857142857142857
🏃 View run Final Model Evaluation at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1/runs/8e4d93eb2b2e423ab53a2f0e5343cced
🧪 View experiment at: https://mlops-25-634030133521.us-west2.run.app/#/experiments/1
